In [1]:
import pandas as pd 
import numpy as np

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
test_data["Fare"].fillna(35, inplace=True)

In [4]:
train_data["Embarked"].fillna("C", inplace=True)

In [5]:
data = [train_data, test_data]
for item in data:
    item.drop(columns="Cabin", inplace=True)

In [6]:
#Calculate mean and standard deviation
data = [train_data, test_data]
for dataset in data:
    mean = dataset["Age"].mean()
    std = dataset["Age"].std()

    #save null values in the age column into a variable 
    isnull = dataset["Age"].isna().sum()

    #generate random integers between standard deviation and mean for null values
    random_age = np.random.randint(mean-std, mean+std, size=isnull)

    #copy data in age column 
    age_slice = dataset["Age"].copy()

    #Let my randomly generated numbers above be fitted into null values in age column 
    age_slice[np.isnan(age_slice)] = random_age


    dataset["Age"] = age_slice
    dataset["Age"] = dataset["Age"].astype(int)

dataset["Age"].isna().sum()

0

In [7]:
# Convert *all datatypes needed for predictive analysis to integers 

data = [train_data, test_data]
for dataset in data:
    #1
    dataset["Fare"] = dataset["Fare"].astype(int)
    #2
    gender = {"male": 0, "female": 1}
    dataset["Sex"] = dataset["Sex"].map(gender)

    #3
    ports = {"S": 1, "C": 2, "Q": 3}
    dataset["Embarked"] = dataset["Embarked"].map(ports)

In [8]:
## MODEL BUILDING 

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [9]:
y_train = train_data["Survived"]
X_train = train_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked"]]
X_test = test_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked"]]
model = DecisionTreeRegressor()
model.fit(X_train,y_train)
test_data["Survived"] = model.predict(X_test)
test_data["Survived"] = test_data["Survived"].astype(int)

In [10]:
test_data[["PassengerId", "Survived"]]

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
